In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time

import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import numpy as np

from IPython.core.display import display,HTML
from geopy.geocoders import ArcGIS

C:\Users\mdima\AppData\Local\Temp\ipykernel_14452\3638562896.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


### quick preparation

In [2]:
def click(x):
    button = driver.find_element(By.XPATH, x)
    button.click()
    time.sleep(2)

preparing the method to get the latitude and longitude coordinates from an address

In [4]:
# preparing stuff for geopy
nom = ArcGIS()

creating paths, which the webdriver runs through. change the range(200) with smaller number to shorten the run.

In [8]:
# defining possible xpath's that we want to try to click later on
# empty list to save multiple xpath from the page
xpath_list = []
# making a list of clickable xpaths
for i in range(200):
    xpath = '/html/body/div[2]/div[2]/main/div/div/div/div[2]/div[4]/div/div/div/div[2]/div/div[%s]/a'%str(i+1)
    xpath_list.append(xpath)

### selenium webdriver: clicking through all state and city names
this is the long-run-part, where it opens the trader joe's website and clicks through all the states and cities.<br>
this might take long. if you wish to make it shorter just to see how it works in general,<br>
reduce the number inside the range(200) in the loop above (you can change it to e.g. 5, it will get only 5 states).

In [10]:
url = "https://locations.traderjoes.com"
# disabling browser popup notifications
options = Options()
options.add_argument("--disable-notifications")
# to prevent from being spotted as a robot
options.add_argument('--disable-gpu')
options.add_argument('user-agent=fake-useragent')
# installing chromedriver, so that we dont need to keep the chromedriver file
# that needs to be updated every once in a while. better install the latest automatically
driver = webdriver.Chrome(ChromeDriverManager().install())
# opens the browser, maximize window size
driver.maximize_window()
# opening url
driver.get(url)
# wait a bit until the "I accept" popup shows
time.sleep(2)
store_list = []
for i in xpath_list:
    try:
        click(i)
        for j in xpath_list:
            try:
                click(j)
                stores = driver.current_url
                store_list.append(stores)
                driver.back()
            except:
                driver.back()
                break
    except:
        break

C:\Users\mdima\AppData\Local\Temp\ipykernel_14452\2879274901.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


check what we obtained.

In [12]:
store_list[0:3]

['https://locations.traderjoes.com/al/birmingham/',
 'https://locations.traderjoes.com/al/huntsville/',
 'https://locations.traderjoes.com/ar/little-rock/']

### beautifulsoup4: getting all details from all city urls
get all informations from each store in the store_list

In [15]:
laddress = []
lcity = []
lstate = []
lzipcode = []
llocation = []
for i in store_list:
    response = requests.get(i)
    contents = response.content
    soup = bs4(contents, "html.parser")
    allres = soup.find_all("div",{"class":"address-left"})
    for j in allres:
        address = j.find_all("span")[1].text
        city = j.find_all("span")[2].text
        state_l = soup.find("div",{"class":"breadcrumb container"}).find_all("a")[2].text
        state_s = j.find_all("span")[3].text
        zipcode = j.find_all("span")[4].text
        country = j.find_all("span")[5].text
        location = address+", "+city+", "+state_s+", "+zipcode+", "+country
        laddress.append(address)
        lcity.append(city)
        lstate.append(state_l)
        lzipcode.append(zipcode)
        llocation.append(location)
df = pd.DataFrame()
df["address"] = laddress
df["city"] = lcity
df["state"] = lstate
df["zipcode"] = lzipcode
df["location"] = llocation
df["coordinates"] = df["location"].apply(nom.geocode)
df["latitude"] = df["coordinates"].apply(lambda x: x.latitude if x != None else None)
df["longitude"] = df["coordinates"].apply(lambda x: x.longitude if x != None else None)

check what the result looks like.

In [16]:
df.head()

,address,city,state,zipcode,location,coordinates,latitude,longitude
0,"205 Summit Blvd, Suite 100",Birmingham,Alabama (AL),35243,"205 Summit Blvd, Suite 100, Birmingham, AL, 35...","(205 Summit Blvd, Ste 100, Vestavia, Alabama, ...",33.447667,-86.728814
1,1100 Mid City Dr NW,Huntsville,Alabama (AL),35806,"1100 Mid City Dr NW, Huntsville, AL, 35806, US","(35806, Huntsville, Alabama, (34.7551700000000...",34.755170,-86.694565
2,11500 Financial Centre Pky,Little Rock,Arkansas (AR),72211,"11500 Financial Centre Pky, Little Rock, AR, 7...","(11500 Financial Centre Pky, Little Rock, Arka...",34.748223,-92.401625
3,1779 E. Williams Field Rd.,Gilbert,Arizona (AZ),85295,"1779 E. Williams Field Rd., Gilbert, AZ, 85295...","(1779 E Williams Field Rd, Gilbert, Arizona, 8...",33.306444,-111.751715
4,7720 West Bell Rd,Glendale,Arizona (AZ),85308,"7720 West Bell Rd, Glendale, AZ, 85308, US","(7720 W Bell Rd, Glendale, Arizona, 85308, (33...",33.639301,-112.226531


In [18]:
print("number of rows:", df.shape[0])

number of rows: 547


beautiful. now save the result to csv file and be proud.

In [17]:
df.to_csv("trader_joes_store_data.csv", index = False)